<a href="https://colab.research.google.com/github/trollmannen/Data-Science/blob/main/WoW-Auction-House-Data-Collector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [1]:
CLIENT_ID = ""
CLIENT_SECRET = ""
REGION = "eu" #"us"
LOCALE = "en_GB" #"en_US"

In [ ]:
#!pip install python-blizzardapi

In [ ]:
from blizzardapi import BlizzardApi
api_client = BlizzardApi(CLIENT_ID, CLIENT_SECRET)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Part 1: Gather Data

In [ ]:
import requests
def create_access_token():
  data = { 'grant_type': 'client_credentials' }
  response = requests.post(f'https://{REGION}.battle.net/oauth/token',
                           data=data, auth=(CLIENT_ID, CLIENT_SECRET))
  return response.json()['access_token']

access_token = create_access_token()

In [ ]:
def get_connected_realms_index(access_token):
  connected_realms_index = {}
  response = requests.get(f'https://{REGION}.api.blizzard.com/data/wow/search/'
                          f'connected-realm?namespace=dynamic-{REGION}'
                          f'&access_token={access_token}')
  for connected_realm in response.json()['results']:
    for realm in connected_realm['data']['realms']:
      connected_realms_index[realm['slug']] = connected_realm['data']['id']

  return pd.Series(connected_realms_index)

connected_realms_index = get_connected_realms_index(access_token)

In [ ]:
def get_auctions(connected_realm_id):
  auction_id,item_id,pet_breed_id,pet_level,pet_quality_id,pet_species_id,unit_price,buyout,quantity,time_left = ([] for i in range(10))
  response = api_client.wow.game_data.get_auctions(REGION,
                                                   LOCALE,
                                                   connected_realm_id)
  for auction in response['auctions']:
    auction_id.append(auction['id'])    
    item_id.append(auction['item']['id'])

    if auction['item']['id'] == 82800:
      pet_breed_id.append(auction['item']['pet_breed_id'])
      pet_level.append(auction['item']['pet_level'])
      pet_quality_id.append(auction['item']['pet_quality_id'])
      pet_species_id.append(auction['item']['pet_species_id'])
    else:
      pet_breed_id.append(0)
      pet_level.append(0)
      pet_quality_id.append(0)
      pet_species_id.append(0)

    if 'unit_price' in auction:
      unit_price.append(auction['unit_price'])
    else:
      unit_price.append(0)

    if 'buyout' in auction:
      buyout.append(auction['buyout'])
    else:
      buyout.append(0)

    quantity.append(auction['quantity'])
    time_left.append(auction['time_left'])

  auctions = pd.DataFrame(list(zip(auction_id,item_id,pet_breed_id,pet_level,pet_quality_id,
                                   pet_species_id,unit_price,buyout,quantity,time_left)),
                          columns=['auction_id','item_id','pet_breed_id','pet_level','pet_quality_id',
                                   'pet_species_id','unit_price','buyout','quantity','time_left'])
  
  auctions['crid'] = connected_realm_id

  return auctions

In [ ]:
all_auctions = pd.DataFrame(columns=['auction_id','item_id','pet_breed_id','pet_level','pet_quality_id',
                                     'pet_species_id','unit_price','buyout','quantity','time_left','crid'])

In [ ]:
from tqdm import tqdm
for connected_realm_id in tqdm(connected_realms_index.unique()):
  all_auctions = pd.concat([get_auctions(connected_realm_id),
                            all_auctions], axis=0)


100%|██████████| 92/92 [03:30<00:00,  2.28s/it]


In [ ]:
def get_item(item_id):
  item = api_client.wow.game_data.get_item(region=REGION,
                                           locale=LOCALE,
                                           item_id=item_id)
  return item

In [ ]:
def get_pet(pet_species_id):
  pet = api_client.wow.game_data.get_pet(region=REGION,
                                         locale=LOCALE,
                                         pet_id=pet_species_id)
  return pet

In [ ]:
all_auctions.to_csv('all_auctions.csv',index=False)

## Part 2: Clean & Organize Data

In [ ]:
all_auctions = pd.read_csv('all_auctions.csv',index_col='auction_id')

In [ ]:
all_auctions['buyout'] = all_auctions['unit_price'] + all_auctions['buyout']

In [ ]:
all_auctions = all_auctions[all_auctions['buyout'] > 0].drop(columns=['unit_price'])

In [ ]:
# todo: fix this monster
all_auctions['gold'] = all_auctions['buyout'].astype(str).str[:-4]
all_auctions['silver'] = all_auctions['buyout'].astype(str).str[-4:-2]
all_auctions['copper'] = all_auctions['buyout'].astype(str).str[-2:]

all_auctions['gold'] = pd.to_numeric(all_auctions['gold'])
all_auctions['silver'] = pd.to_numeric(all_auctions['silver'])
all_auctions['copper'] = pd.to_numeric(all_auctions['copper'])

all_auctions['gold'] = all_auctions['gold'].fillna(0)
all_auctions['silver'] = all_auctions['silver'].fillna(0)
all_auctions['copper'] = all_auctions['copper'].fillna(0)

all_auctions['gold'] = all_auctions['gold'].astype('int')
all_auctions['silver'] = all_auctions['silver'].astype('int')
all_auctions['copper'] = all_auctions['copper'].astype('int')

In [ ]:
from datetime import datetime
all_auctions['timestamp'] = datetime.now()
#all_auctions = all_auctions.set_index('timestamp')

In [ ]:
item_index = pd.read_csv('/content/drive/MyDrive/wow-ah/item_index.csv',
                         index_col='id',squeeze=True)
pet_index = pd.read_csv('/content/drive/MyDrive/wow-ah/pet_index.csv',
                        index_col='id',squeeze=True)

def index(ids,index,get_x):
  for id in tqdm(ids):
    if id in index:
      continue
    x = get_x(id)
    try:
      index[id] = x['name']
    except KeyError:
      index[id] = pd.NA

index(all_auctions['item_id'].unique(),item_index,get_item)
index(all_auctions['pet_species_id'].unique(),pet_index,get_pet)

item_index.to_csv('/content/drive/MyDrive/wow-ah/item_index.csv')
pet_index.to_csv('/content/drive/MyDrive/wow-ah/pet_index.csv')

100%|██████████| 802/802 [00:00<00:00, 114326.61it/s]


In [ ]:
def set_name(item_id,pet_id):
  if item_id == 82800:
    return pet_index[pet_id]
  return item_index[item_id]

all_auctions['name'] = np.vectorize(set_name)(all_auctions['item_id'],
                                              all_auctions['pet_species_id'])

In [ ]:
all_auctions['epoch_hours'] = all_auctions['timestamp'].view('int64')//1e9//3600
all_auctions['epoch_hours'] = all_auctions['epoch_hours'].astype('int')

## Part 3: Store Data

In [ ]:
#rolling_window = []

In [ ]:
rolling_window.insert(0,all_auctions)
if len(rolling_window) > 2:
  rolling_window.pop()

In [ ]:
len(rolling_window[0])

4363148

In [ ]:
len(rolling_window[-1])

4328639

In [ ]:
if len(rolling_window) > 1:
  new = set(rolling_window[0].index)
  old_no_short = set(rolling_window[-1][rolling_window[-1]['time_left'] != 'SHORT'].index)
  old_only_short = set(rolling_window[-1][rolling_window[-1]['time_left'] == 'SHORT'].index)
  old = set(rolling_window[-1].index)

  sold_auctions = list(sorted(old_no_short - new))
  new_auctions = list(sorted(new - old))
  exceeded_auctions = list(sorted(old_only_short - new))

In [ ]:
len(sold_auctions)

161534

In [ ]:
len(new_auctions)

266459

In [ ]:
len(exceeded_auctions)

70416

In [ ]:
sold = rolling_window[-1][rolling_window[-1].index.isin(sold_auctions)]

In [ ]:
sold.groupby('pet_species_id').count().nlargest(10,'gold')

,item_id,pet_breed_id,pet_level,pet_quality_id,buyout,quantity,time_left,crid,gold,silver,copper,timestamp,name,epoch_hours
pet_species_id,,,,,,,,,,,,,,
0,150796,150796,150796,150796,150796,150796,150796,150796,150796,150796,150796,150796,150796,150796
2916,92,92,92,92,92,92,92,92,92,92,92,92,92,92
200,86,86,86,86,86,86,86,86,86,86,86,86,86,86
3064,82,82,82,82,82,82,82,82,82,82,82,82,82,82
3044,64,64,64,64,64,64,64,64,64,64,64,64,64,64
122,61,61,61,61,61,61,61,61,61,61,61,61,61,61
2468,51,51,51,51,51,51,51,51,51,51,51,51,51,51
3006,49,49,49,49,49,49,49,49,49,49,49,49,49,49
1151,46,46,46,46,46,46,46,46,46,46,46,46,46,46


In [ ]:
get_item(2916)['name']

'Desolate Leather'

In [ ]:
get_pet(2916)['name']

'Hungry Burrower'

In [ ]:
all_auctions[all_auctions['pet_species_id']==2463]

,item_id,pet_breed_id,pet_level,pet_quality_id,pet_species_id,buyout,quantity,time_left,crid,gold,silver,copper,timestamp,name,epoch_hours
auction_id,,,,,,,,,,,,,,,
1439739810,82800,9,1,3,2463,10000000,1,SHORT,1127,1000,0,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
1439761349,82800,9,1,3,2463,7374100,1,MEDIUM,1127,737,41,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
1440019913,82800,9,1,3,2463,7374100,1,LONG,1127,737,41,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
1440112217,82800,9,25,3,2463,111780000,1,VERY_LONG,1127,11178,0,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
1440112220,82800,9,25,3,2463,111780000,1,VERY_LONG,1127,11178,0,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341159662,82800,9,25,3,2463,9979900,1,LONG,3686,997,99,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
55114438,82800,9,1,3,2463,3000000,1,MEDIUM,1623,300,0,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608
56065039,82800,9,25,3,2463,79990000,1,VERY_LONG,1623,7999,0,0,2022-04-26 16:06:40.423672,Sandshell Chitterer,458608


In [ ]:
len(all_auctions)

4363148

In [ ]:
len(all_auctions[all_auctions['item_id']==82800])

398851